In [1]:
import pandas as pd
#read in the 2012 data
data_2012 = pd.read_csv('2012.csv')

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_2012.head()

,Unnamed: 0,EMP,ESTAB,NAICS2012_TTL,OPTAX,GEO_TTL,NAICS2012,state,place
0,0,874,63,Total for all sectors,99,"Ituau county, American Samoa",00,60,37700
1,1,4344,164,Total for all sectors,99,"Ma'oputasi county, American Samoa",00,60,51300
2,2,0,5,Total for all sectors,99,"Sa'ole county, American Samoa",00,60,67300
3,3,0,4,Total for all sectors,99,"Sua county, American Samoa",00,60,72100
4,4,0,1,Total for all sectors,99,"Vaifanua county, American Samoa",00,60,86500


In [5]:
#Collect the state names and abbreviation data
!curl https://raw.githubusercontent.com/fitnr/addfips/master/addfips/data/states.csv?accessType = Download > states.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1036  100  1036    0     0   3149      0 --:--:-- --:--:-- --:--:--  3158
curl: (6) Could not resolve host: =
curl: (6) Could not resolve host: Download


In [6]:
#turn the states.csv into a dataframe
states = pd.read_csv('states.csv')

In [7]:
states.head()

,name,postal,fips
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6


In [8]:
#Turn FIPs codes into strings
states['fips'] = states['fips'].astype(str)
data_2012['state'] = data_2012['state'].astype(str)

In [9]:
#Fill the fips with the extra zeros needed for joining
states['fips'] = states.fips.str.zfill(2)
data_2012['state'] = data_2012.state.str.zfill(2)

In [10]:
#select necessary columns
states_cleaned = states[['postal','fips']]

In [11]:
states_cleaned.head()

,postal,fips
0,AL,01
1,AK,02
2,AZ,04
3,AR,05
4,CA,06


In [12]:
#join the states and 2012 data
data_2012_us = data_2012.merge(states_cleaned, left_on = 'state', right_on = 'fips', how = 'inner')

In [13]:
data_2012_us.head()

,Unnamed: 0,EMP,ESTAB,NAICS2012_TTL,OPTAX,GEO_TTL,NAICS2012,state,place,postal,fips
0,0,874,63,Total for all sectors,99,"Ituau county, American Samoa",00,60,37700,AS,60
1,1,4344,164,Total for all sectors,99,"Ma'oputasi county, American Samoa",00,60,51300,AS,60
2,2,0,5,Total for all sectors,99,"Sa'ole county, American Samoa",00,60,67300,AS,60
3,3,0,4,Total for all sectors,99,"Sua county, American Samoa",00,60,72100,AS,60
4,4,0,1,Total for all sectors,99,"Vaifanua county, American Samoa",00,60,86500,AS,60


In [14]:
#Get territory fips
not_us_states = ['60','66','69','72','78']

In [15]:
#remove territories 
data_2012_states = data_2012_us.loc[~data_2012_us['fips'].isin(not_us_states)]

In [16]:
#Remove county balances
data_2012_cities = data_2012_states[~data_2012_states['GEO_TTL'].str.contains('Balance')]

In [17]:
#define tax codes of interest
tax_codes = ['99','A']

In [18]:
#Filter data to said tax codes
data_2012_cities_all_occupation_1 = data_2012_cities.loc[data_2012_cities['OPTAX'].isin(tax_codes)]

In [19]:
#Drop duplicates
data_2012_cities_remove_dupes = data_2012_cities_all_occupation_1.drop_duplicates() 

In [21]:
#Begin to clean city name to split on city, CPD, or town. For instance we don't want New York City city, just New York City
data_2012_cities_remove_dupes['city_name'] = pd.np.where(data_2012_cities_remove_dupes.GEO_TTL.str.contains("city"),
                                                         data_2012_cities_remove_dupes.GEO_TTL.str.split("city"),
                                             pd.np.where(data_2012_cities_remove_dupes.GEO_TTL.str.contains("CDP"),
                                                         data_2012_cities_remove_dupes.GEO_TTL.str.split("CDP"),
                                             pd.np.where(data_2012_cities_remove_dupes.GEO_TTL.str.contains("town"),
                                                         data_2012_cities_remove_dupes.GEO_TTL.str.split("town"), 
                                                         "task")))

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
#select only the city name
data_2012_cities_remove_dupes['city_name'] = data_2012_cities_remove_dupes['city_name'].str[0]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
#Create the unique city_state code
data_2012_cities_remove_dupes['city_state_code'] = data_2012_cities_remove_dupes['city_name'] + '_' + data_2012_cities_remove_dupes['postal']

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
#Get a list of all unique NAICS titles
jobs = list(data_2012['NAICS2012_TTL'].unique())

In [26]:
#Select all job titles with certain creative class
for i in jobs:
    if 'engineering' in i:
        print(i)
    if 'science' in i:
        print(i)
    if 'lawyer' in i:
        print(i)
    if 'medical' in i:
        print(i)
    if 'medicine' in i:
        print(i)
    if 'education' in i:
        print(i)    
    if 'computer' in i:
        print(i)     
    if 'accounting' in i:
        print(i)    
    if 'engineer' in i:
        print(i)
    if 'tax' in i:
        print(i)
    if 'technology' in i:
        print(i)
    if 'programming' in i:
        print(i)
    if 'artists' in i:
        print(i)
    if 'music' in i:
        print(i)
    if 'bio' in i:
        print(i)
    if 'research' in i:
        print(i)
    if 'legal' in i:
        print(i)
    if 'development' in i:
        print(i)
    if 'planning' in i:
        print(i)
    if 'surgical' in i:
        print(i)

Heavy and civil engineering construction
Heavy and civil engineering construction
Veneer, plywood, and engineered wood product manufacturing
Pharmaceutical and medicine manufacturing
Electronic computer manufacturing
Computer terminal and other computer peripheral equipment manufacturing
Navigational, measuring, electromedical, and control instruments manufacturing
Electromedical and electrotherapeutic apparatus manufacturing
Surgical and medical instrument manufacturing
Computer and computer peripheral equipment and software merchant wholesalers
Computer and computer peripheral equipment merchant wholesalers
Surgical, medical, and hospital supplies merchant wholesalers
Sporting goods, hobby, musical instrument, and book stores
Sporting goods, hobby, and musical instrument stores
Other water passenger transportation (including water taxi)
Cable and other subscription programming
Direct life, health, and medical insurance carriers
Direct health and medical insurance carriers
Direct insu

In [27]:
#create a list of job tittles that would be a part of the creative class
creative_class = ['Heavy and civil engineering construction',
'Veneer, plywood, and engineered wood product manufacturing',
'Pharmaceutical and medicine manufacturing',
'Electronic computer manufacturing',
'Computer terminal and other computer peripheral equipment manufacturing',
'Navigational, measuring, electromedical, and control instruments manufacturing',
'Electromedical and electrotherapeutic apparatus manufacturing',
'Surgical and medical instrument manufacturing',
'Computer and computer peripheral equipment and software merchant wholesalers',
'Computer and computer peripheral equipment merchant wholesalers',
'Surgical, medical, and hospital supplies merchant wholesalers',
'Direct life, health, and medical insurance carriers',
'Direct health and medical insurance carriers',
'Direct insurance (except life, health, and medical) carriers',
'Offices of lawyers',
'Legal aid societies and similar legal services',
'Other legal services',
'All other legal services',
'Accounting, tax preparation, bookkeeping, and payroll services',
'Other accounting services',
'Architectural, engineering, and related services',
'Custom computer programming services',
'Other computer related services',
'Scientific research and development services',
'Research and development in the physical, engineering, and life sciences',
'Research and development in the physical, engineering, and life sciences',
'Research and development in biotechnology',
'Research and development in the physical, engineering, and life sciences (except biotechnology)',
'Research and development in the physical and engineering sciences',
'Research and development in other life sciences',
'Research and development in the social sciences and humanities',
'Marketing research and public opinion polling',
'Economic or industrial planning or development organization',
'Business schools and computer and management training',
'Professional and management development training',
'Art, drama, and music schools',
'Family planning centers',
'HMO medical centers',
'Freestanding ambulatory surgical and emergency centers',
'General medical and surgical hospitals',
'General medical and surgical hospitals, government',
'General medical and surgical hospitals, except government',
'Residential intellectual and developmental disability, mental health, and substance abuse facilities',
'Residential intellectual and developmental disability facilities',
'Musical groups and artists',
'Symphony orchestras and chamber music organizations',
'Other music groups and artists',
'Agents and managers for artists, athletes, entertainers, and other public figures',
'Independent artists, writers, and performers']

In [28]:
#case when clause to define which jobs and establishments would be labeled as creative
data_2012_cities_remove_dupes['class'] = pd.np.where(
     data_2012_cities_remove_dupes['NAICS2012_TTL'].isin(creative_class), 
    '1_creative',  
    pd.np.where(~data_2012_cities_remove_dupes['NAICS2012_TTL'].isin(creative_class), 
    '2_regular','3_none'
     ))

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
#QA to determine that everything is either regular or creative
data_2012_cities_remove_dupes['class'].unique()

array(['2_regular', '1_creative'], dtype=object)

In [30]:
#reset the index
data_2012_cleaned_ = data_2012_cities_remove_dupes.reset_index()

In [31]:
#save this datset as semi processed (will be needed for 2002 and 2007 years without NAICS tittles)
data_2012_cleaned_.to_csv('2012_semi_processed.csv')

In [32]:
#select columns of interest
data_2012_cleaned_ = data_2012_cleaned_af[['city_state_code','EMP','ESTAB','class']]

In [34]:
#Groupby by and unstack by class
econ_data_final = data_2012_cleaned_af.groupby(['city_state_code','class']).sum().unstack()

In [36]:
#reset the index and rename the columns
econ_data_final = econ_data_final.reset_index()
econ_data_final.columns = ['city_state','creative_employees','regular_employees',
                           'creative_establishments','regular_establishments']

In [40]:
#write out the dataframe to csv
econ_data_final.to_csv('2012_creative_class.csv')